In [45]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [46]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\gargi
[nltk_data]     saha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [47]:
nltk.data.path.append('C:\\Users\\gargi saha\\nltk_data')
nltk.download('punkt', download_dir='C:\\Users\\gargi saha\\nltk_data')

[nltk_data] Downloading package punkt to C:\Users\gargi
[nltk_data]     saha\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [48]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read() # read json file
intents = json.loads(data_file)

In [49]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Simple tokenization (splitting by spaces)
        w = pattern.split()
        words.extend(w)  # Add words to the list
        documents.append((w, intent['tag']))  # Store words with intent tag
        
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
documents

[(['Hi', 'there'], 'greeting'),
 (['How', 'are', 'you'], 'greeting'),
 (['Is', 'anyone', 'there?'], 'greeting'),
 (['Hey'], 'greeting'),
 (['Hola'], 'greeting'),
 (['Hello'], 'greeting'),
 (['Good', 'day'], 'greeting'),
 (['Bye'], 'goodbye'),
 (['See', 'you', 'later'], 'goodbye'),
 (['Goodbye'], 'goodbye'),
 (['Nice', 'chatting', 'to', 'you,', 'bye'], 'goodbye'),
 (['Till', 'next', 'time'], 'goodbye'),
 (['Thanks'], 'thanks'),
 (['Thank', 'you'], 'thanks'),
 (["That's", 'helpful'], 'thanks'),
 (['Awesome,', 'thanks'], 'thanks'),
 (['Thanks', 'for', 'helping', 'me'], 'thanks'),
 (['Help', 'me'], 'help'),
 (['I', 'need', 'assistance'], 'help'),
 (['Can', 'you', 'help?'], 'help'),
 (["I'm", 'stuck'], 'help'),
 (['Thanks'], 'thanks'),
 (['Thank', 'you'], 'thanks'),
 (['Thanks', 'a', 'lot'], 'thanks'),
 (['Tell', 'me', 'about', 'tourist', 'spots'], 'tourist_spots'),
 (['What', 'are', 'the', 'famous', 'places', 'in', 'Kolkata?'],
  'tourist_spots'),
 (['Tourist', 'attractions'], 'tourist_spo

In [50]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\gargi
[nltk_data]     saha/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [51]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to C:\Users\gargi
[nltk_data]     saha/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [52]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents\n", documents, "\n")
# classes = intents[tag]
print (len(classes), "classes\n", classes, "\n")
# words = all words, vocabulary
print (len(words), "unique lemmatized words\n", words, "\n")
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

1081 documents
 [(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you,', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (["That's", 'helpful'], 'thanks'), (['Awesome,', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['Help', 'me'], 'help'), (['I', 'need', 'assistance'], 'help'), (['Can', 'you', 'help?'], 'help'), (["I'm", 'stuck'], 'help'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['Thanks', 'a', 'lot'], 'thanks'), (['Tell', 'me', 'about', 'tourist', 'spots'], 'tourist_spots'), (['What', 'are', 'the', 'famous', 'places', 'in', 'Kolkata?'], 'tourist_spots'), (['Tourist', 'attractions'], 'tourist_spots'), (['Tel

In [53]:
import random
import numpy as np
from nltk.stem import WordNetLemmatizer

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Create training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    # Create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # Create output row
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# Shuffle and convert to NumPy array
random.shuffle(training)
training = np.array(training, dtype=object)  # ✅ Fix applied

# Create train and test data
train_x = list(training[:, 0])
train_y = list(training[:, 1])

print("Training data created")

Training data created


In [54]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to C:\Users\gargi
[nltk_data]     saha/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [55]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [56]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
print("First layer:",model.layers[0].get_weights()[0])

First layer: [[ 6.9789231e-02  5.3997859e-02  7.3883429e-02 ...  2.8091125e-02
   7.0653886e-02 -6.4157903e-02]
 [-7.6191083e-02  5.9587479e-02  7.8641012e-02 ...  5.0851569e-02
   2.7292468e-02 -4.6820324e-02]
 [-4.9448166e-02 -4.7476441e-03 -2.0396855e-02 ...  2.2944666e-02
   2.8674379e-03  4.2323552e-02]
 ...
 [-7.0057161e-02 -1.3157561e-02  1.5323155e-02 ...  2.2476047e-02
  -8.8402480e-03  3.0169033e-02]
 [ 7.8167662e-02  9.3847513e-05  7.2917134e-02 ...  6.1372548e-02
   6.9950715e-02 -2.4374604e-02]
 [ 2.6480906e-02 -4.9077202e-02  4.2331129e-02 ...  5.1221147e-02
   4.2586170e-02  1.9617230e-02]]


In [57]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [58]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


Epoch 1/200
217/217 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0018 - loss: 5.1371
Epoch 2/200
217/217 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0251 - loss: 4.9512
Epoch 3/200
217/217 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0311 - loss: 4.7303
Epoch 4/200
217/217 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0733 - loss: 4.4438
Epoch 5/200
217/217 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1020 - loss: 4.0970
Epoch 6/200
217/217 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1551 - loss: 3.8375
Epoch 7/200
217/217 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1614 - loss: 3.6003
Epoch 8/200
217/217 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2481 - loss: 3.3489
Epoch 9/200
217/217 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2148 - loss: 3.1809
Epoch 10/200
217/217 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2737 - loss: 2.9557
Epoch 11/200
217/217 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3059 - loss: 2.7080
Epoch 12/200
217/217 ━━━━━━━━━━━━━━━━━━━━

model created
